# Create LULC

In [5]:
%matplotlib inline
%load_ext chime

import os
from glob import glob
import pickle
import chime

import rasterio as rio
import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage.morphology import binary_closing, binary_opening, square, remove_small_holes, disk, square

from skimage.filters.rank import modal, mean_bilateral
from datetime import datetime, timedelta, time
from time import sleep
import nbimporter
#from KeyFunctions import *
import chime
from utils import *


def throttleProcessors(workDayStart, workDayEnd):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause or nowTime > startPause:
        print(f"Current time is {nowTime}. Setting processor/thread use to 4")
        return 4
    else:
        return 8

def wait_start(workDayStart="8:30", workDayEnd="18:00", force=False):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause and nowTime > startPause and datetime.today().isoweekday() <= 5 and not force:
        waitTimeSeconds = datetime.combine(datetime.today(), endPause) - datetime.combine(datetime.today(), nowTime)
    else:
        waitTimeSeconds = False
    return waitTimeSeconds


def createClassifiedRaster(classification_model, ortho_file, classifiedFiles_loc, binaryClass=None, overwrite=False):
    start = datetime.now()
    day = datetime.now().strftime("%Y%m%d")
    daynum = 1
    output_image = os.path.join(classifiedFiles_loc, os.path.basename(ortho_file))#_{day}-{daynum}.tif"))
    if binaryClass:
        #print(f"Setting binary output ({output_image})")
        output_image = output_image.replace(".tif", f"_{binaryClass}BinaryLGBNewishGBLM.tif")
    if os.path.exists(output_image) and not overwrite:
        #print(f"File exists ({output_image})")
        return output_image
    try:
        with rio.open(ortho_file) as src:
            kwargs = src.profile
            kwargs.update(
                dtype= rio.uint8,
                count= 1,
            )

            data = src.read()
            descs = list(src.descriptions)
    except:
        print(f"ERROR: Unable to open {os.path.basename(ortho_file)}. Skipping")
        return None
    
    kwargs.update(
        dtype= rio.uint8,
        count= 1,
    )
    
    try:
        feature_names = classification_model.feature_name_
    except:
        feature_names = classification_model.feature_name()
    

    bands = {desc:data[ib] for ib, desc in enumerate(descs)}
    
    #This is a training dataset not created with the others, but may be in the classifier. Create and add
    if "RGBNmean" in feature_names:
        #print("Adding RGBNmean")
        rgbnMean = np.nanmean(data[:4], axis=0).astype(data.dtype)
        bands["RGBNmean"] = rgbnMean

    features = {fn:bands[fn] for fn in feature_names}# if fn in descs}
    
    featureArrays = list(features.values())
            
    trainingFeatures = np.stack(featureArrays)

    #print("Input Data shape", data.shape)
    
    # read the image into the proper format, adding indices if necessary
    img_swp = np.moveaxis(trainingFeatures, 0, -1)
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])
    
    #flatten bands along axis
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])
    
    # remove no data values, store the indices for later use
    # a later cell makes the assumption that all bands have identical no-data value arrangements
    m = np.ma.masked_invalid(img_flat)
    to_predict = img_flat[~m.mask].reshape(-1, img_flat.shape[-1])
    #print("TO PREDICT SHAPE", to_predict.shape)
    
    # predict
    #print("Beginning prediction...", datetime.now())
    img_preds = classification_model.predict(to_predict)
    
    # add the prediction back to the valid pixels (using only the first band of the mask to decide on validity)
    # resize to the original image dimensions
    output = np.zeros(img_flat.shape[0])
    output[~m.mask[:,0]] = img_preds.flatten()
    output = output.reshape(*img_swp.shape[:-1])
    
    output = np.rint(output)
    # create our final mask
    #mask = (~m.mask[:,0]).reshape(*img_swp.shape[:-1])
    
    with rio.open(output_image, 'w', **kwargs) as dst: 

        colors = {
            1: (12,42,235, 255),
            2: (41, 210, 219,255),
            3: (255, 214, 117, 255),
            4: (171, 224, 85, 255),
            5: (12, 100, 1, 255),
            6: (0, 192, 32, 255),
            7: (62, 62, 62, 255),
            8: (160, 160, 160, 255),
            9: (160, 37, 6, 255)
        }
        
        # write to the final file
        dst.write(output.astype(rio.uint8), 1)
        #dst.write_mask(mask)
        
        dst.write_colormap(1, colors)
    
    print(f"Classified to {os.path.abspath(output_image)}. \nClassification took {datetime.now()-start}")
    
    return output_image

    %chime
    
    
def classifyTiles(df, forceStart=False, overwrite=False, binaryClass=None):
    print(f"\n\nStarting processing for {len(df)} tiles\n")
    for i, r in df.iterrows():
        #bestlgbm_model.set_params(n_jobs=throttleProcessors("8:30", "18:00"))
        waitTime = wait_start(force=forceStart)
        if waitTime:
            buildVRT(classifiedFiles_loc, "EPC_30cmOrthoSegmented_Classified.vrt")
            pause_until = datetime.now() + waitTime
            print(f"waiting {timedelta(seconds=waitTime.seconds)} until {pause_until.strftime('%H:%m')}")
            sleep(waitTime.seconds)
            
        print(f"Starting {r.path}_{r.row} @ {datetime.now()}")
        try:
            result = createClassifiedRaster(classification_model=bestlgbm_model,
                                            ortho_file=r.OrthoFile,
                                            classifiedFiles_loc=classifiedFiles_loc,
                                            binaryClass = binaryClass,
                                            overwrite=overwrite)
        except:
            print(f"FAILED FOR {r.OrthoFile}")

    return result

In [6]:
classifiedFiles_loc = "../EPCExtent_30cm/Orthos_Segmented_Classifiedv3"
#propsDir = r"../EPCExtent_30cm/Orthos_Segmentedv2_properties"
orthosDir = r"../EPCExtent_30cm/Orthos_Segmentedv3"
os.makedirs(classifiedFiles_loc, exist_ok=True)

tindex = gpd.read_file("../EPCExtent_30cm/Ortho_5kSubIndex.gpkg")
to_process = gpd.read_file("../temp/ToProcess.gpkg")

tindex["OrthoFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, directory=orthosDir), axis=1)
#tindex["PropsFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, files=propsDir), axis=1)

# ignore tiles which don't have input variables created
tindex = tindex[(~pd.isnull(tindex.OrthoFile))]

#prioritize central tucson and work out from there
tindex["centroid"] = tindex.geometry.centroid
central_tile = tindex[(tindex.path == "W1004789") & (tindex.row == "W449850")]
central_point = central_tile.centroid.values[0]

tindex["DistToCenter"] = tindex.centroid.apply(lambda c: int(c.distance(central_point)))
tindex.sort_values(by="DistToCenter", inplace=True)

#filter out files that are completed
#tindex = tindex[pd.isnull(tindex["Done"])]

#tindex["Done"]= tindex.apply(lambda r: findFile(path=r.path, row=r.row, directory=classifiedFiles_loc),axis=1)
#toProcess = tindex[tindex.geometry.intersects(to_process.unary_union)].reset_index()                                 
#classifyTiles(toProcess)
#finished()
#classifyTiles(tindex)
#finished()

print("\nFINISHED")

C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:294: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for f in features_lst:



FINISHED


In [7]:
classifiers = {"Asphault": "Models/lightGBMBinaryAsphault_20210727.sav",
              "Structure": "Models/lightGBMBinaryStructure_20210630.sav"}

In [8]:
%%time

for lcClass, model_loc in classifiers.items():
    if lcClass != "Asphault":
        continue
    print(lcClass)
    bestlgbm_model = pickle.load(open(model_loc, 'rb'))
    
    tt = tindex[((tindex.path == "W989789") & (tindex.row == "W439850")) |
                ((tindex.path == "W1004789") & (tindex.row == "W449850"))]
    t = classifyTiles(tt, forceStart=True, binaryClass=lcClass, overwrite=True)
    
    

Asphault


Starting processing for 2 tiles

Starting W1004789_W449850 @ 2021-07-28 16:41:20.906856
Classified to M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W1004789_W449850_TrainingStackV3_AsphaultBinaryLGBNewishGBLM.tif. 
Classification took 0:00:31.318220
Starting W989789_W439850 @ 2021-07-28 16:41:52.633658
Classified to M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W989789_W439850_TrainingStackV3_AsphaultBinaryLGBNewishGBLM.tif. 
Classification took 0:00:30.865162
Wall time: 1min 3s


In [9]:
%chime

In [58]:
tarray = t[1]
    tkwargs = t[2]
    print(tkwargs)

    tkwargs.update(dtype=tarray.dtype)
    with rio.open("test.tif", "w", **tkwargs) as dst:
        dst.write(tarray,1)
    finished()



Starting processing for 2 tiles

Starting W1004789_W449850 @ 2021-06-16 13:46:36.417918
Adding RGBNmean
TO PREDICT SHAPE (25796241, 25)
Starting W989789_W439850 @ 2021-06-16 13:47:55.627220
Adding RGBNmean
TO PREDICT SHAPE (25796241, 25)
Wall time: 2min 37s


In [49]:
buildVRT("../UrbanExtent_15cm/Elevation_40cmNPS/DSM40cm", "DSM40cm2019.vrt")

Created M:\PAG2019\UrbanExtent_15cm\Elevation_40cmNPS\DSM40cm\DSM40cm2019.vrt


In [ ]:
def cleanupBinary(array, binaryClass=None):
    if binaryClass == "Structure":
        out[(out==9) & (bands["HAG"]<5)] = 0 # if classified as structure, but less than 5 feet high, reclass to impervious
        #out[(bands["REDness"]>=35336) & (bands["GREENness"]<=34307) & (bands["BLUEness"]<=33409) & (lulc!=9) & (bands["DPR"]<=3)] = 3 # good for red bare earth
        out[(out==33023) & (bands["BLUEness"]<38547) & (bands["NIRness"]>23769)] = 0 # remove pools that aren't very blue AND does not have low NIRness value
            
        out[(bands["MSAVI"]>=29556) & (bands["GREENness"]>=34307) & (bands["RGBNmean"]<=19273) ] = 0 # set very green veg with high index and low brightness to irrigated
        
        out = smoothStructures(out) # smooth structures